Whisper Finetuning on Google Fleurs ASR dataset

#Necessary Installations
Includes login, token, evaluate and transformers install and config. Also included accelerator installation for seq2seqtrainerargs.

In [ ]:
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
import huggingface_hub
huggingface_hub.login(token='XXX') #insert huggingface token here

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 8.0 MB/s eta 0:00:00


#Analyzing the dataset
Since ASR datasets can be pretty huge, it was first important to get a gist of the dataset and understand the parameters. <br>
1. get_dataset_config_names: I used this to get an overview of different languages supported by fleurs and the splits.
2. I used an iterable dataset (load dataset with streaming=True) as to not downlaod the entire dataset, however as I went further syntax-wise, it made more sense to use the entire dataset.


In [ ]:
import datasets

In [ ]:
config = datasets.get_dataset_config_names("google/fleurs")
config

['af_za',
 'am_et',
 'ar_eg',
 'as_in',
 'ast_es',
 'az_az',
 'be_by',
 'bg_bg',
 'bn_in',
 'bs_ba',
 'ca_es',
 'ceb_ph',
 'ckb_iq',
 'cmn_hans_cn',
 'cs_cz',
 'cy_gb',
 'da_dk',
 'de_de',
 'el_gr',
 'en_us',
 'es_419',
 'et_ee',
 'fa_ir',
 'ff_sn',
 'fi_fi',
 'fil_ph',
 'fr_fr',
 'ga_ie',
 'gl_es',
 'gu_in',
 'ha_ng',
 'he_il',
 'hi_in',
 'hr_hr',
 'hu_hu',
 'hy_am',
 'id_id',
 'ig_ng',
 'is_is',
 'it_it',
 'ja_jp',
 'jv_id',
 'ka_ge',
 'kam_ke',
 'kea_cv',
 'kk_kz',
 'km_kh',
 'kn_in',
 'ko_kr',
 'ky_kg',
 'lb_lu',
 'lg_ug',
 'ln_cd',
 'lo_la',
 'lt_lt',
 'luo_ke',
 'lv_lv',
 'mi_nz',
 'mk_mk',
 'ml_in',
 'mn_mn',
 'mr_in',
 'ms_my',
 'mt_mt',
 'my_mm',
 'nb_no',
 'ne_np',
 'nl_nl',
 'nso_za',
 'ny_mw',
 'oc_fr',
 'om_et',
 'or_in',
 'pa_in',
 'pl_pl',
 'ps_af',
 'pt_br',
 'ro_ro',
 'ru_ru',
 'sd_in',
 'sk_sk',
 'sl_si',
 'sn_zw',
 'so_so',
 'sr_rs',
 'sv_se',
 'sw_ke',
 'ta_in',
 'te_in',
 'tg_tj',
 'th_th',
 'tr_tr',
 'uk_ua',
 'umb_ao',
 'ur_pk',
 'uz_uz',
 'vi_vn',
 'wo_sn',
 'xh

In [ ]:
# ds_stream = load_dataset("google/fleurs", "hi_in", split="train")

In [ ]:
from datasets import interleave_datasets, load_dataset

def load_streaming_dataset(dataset_name, dataset_config_name, split, **kwargs):
    if "+" in split:
        # load multiple splits separated by the `+` symbol *with* streaming mode
        dataset_splits = [load_dataset(dataset_name, dataset_config_name, split=split_name, streaming=False, **kwargs) for split_name in split.split("+")]
        # interleave multiple splits to form one dataset
        interleaved_dataset = interleave_datasets(dataset_splits)
        return interleaved_dataset
    else:
        # load a single split *with* streaming mode
        dataset = load_dataset(dataset_name, dataset_config_name, split=split, streaming=False, **kwargs)
        return dataset

In [ ]:
raw_data = datasets.IterableDatasetDict()
raw_data["train"] = load_streaming_dataset("google/fleurs", "hi_in", split="train")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset fleurs downloaded and prepared to /root/.cache/huggingface/datasets/google___fleurs/hi_in/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac. Subsequent calls will reuse this data.


#Sampling
1. I looked at features to see the sampling rate of the dataset. Since the sampling rate of the dataset and the model should match, this was necessary to see if upsampling or downsampling was required. However, both Whisper-medium and fleurs have the same sampling rate.
2. Since whisper computes on 30-second inputs of log-mel spectrograms to be passed into encoders, it was important to keep files strictly less than 30 seconds.
3. I used WhisperProcessor to perform feature extraction and tokenize the generated text.


In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="hindi", task="transcribe")

In [ ]:
raw_data["train"].features


{'id': Value(dtype='int32', id=None),
 'num_samples': Value(dtype='int32', id=None),
 'path': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'transcription': Value(dtype='string', id=None),
 'raw_transcription': Value(dtype='string', id=None),
 'gender': ClassLabel(names=['male', 'female', 'other'], id=None),
 'lang_id': ClassLabel(names=['af_za', 'am_et', 'ar_eg', 'as_in', 'ast_es', 'az_az', 'be_by', 'bg_bg', 'bn_in', 'bs_ba', 'ca_es', 'ceb_ph', 'ckb_iq', 'cmn_hans_cn', 'cs_cz', 'cy_gb', 'da_dk', 'de_de', 'el_gr', 'en_us', 'es_419', 'et_ee', 'fa_ir', 'ff_sn', 'fi_fi', 'fil_ph', 'fr_fr', 'ga_ie', 'gl_es', 'gu_in', 'ha_ng', 'he_il', 'hi_in', 'hr_hr', 'hu_hu', 'hy_am', 'id_id', 'ig_ng', 'is_is', 'it_it', 'ja_jp', 'jv_id', 'ka_ge', 'kam_ke', 'kea_cv', 'kk_kz', 'km_kh', 'kn_in', 'ko_kr', 'ky_kg', 'lb_lu', 'lg_ug', 'ln_cd', 'lo_la', 'lt_lt', 'luo_ke', 'lv_lv', 'mi_nz', 'mk_mk', 'ml_in', 'mn_mn', 'mr_in', 'ms_my', 'mt_mt', 'my_mm', 'nb

Sampling rate is a uniform 16000, so there's no need to upsample or downsample.<br>
1. batch["audio"] loads the audio data.
2. feature_extractor extracts log-mel features from input array
3. batch["transcription"] loads the transcription from the dataset
4. tokenizer converts the transcriptions to labels

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # compute log-Mel features
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]
    transcription = batch["transcription"]

    batch["labels"] = processor.tokenizer(transcription).input_ids
    return batch

In [ ]:
vectorized_datasets = raw_data.map(prepare_dataset, remove_columns=list(next(iter(raw_data.values())).features)).with_format("torch")


Map:   0%|          | 0/2120 [00:00<?, ? examples/s]

In [ ]:
vectorized_datasets["train"] = vectorized_datasets["train"].shuffle(
    seed=0
)

In [ ]:
max_input_length = 30.0

def is_audio_in_length_range(length):
    return length < max_input_length

In [ ]:
vectorized_datasets["train"] = vectorized_datasets["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

Filter:   0%|          | 0/2120 [00:00<?, ? examples/s]

#Training and Evaluation
1. <u>Seq2SeqTrainer</u> helps make the training pipeline
2. <u>Data Collator</u> helps make pytorch tensors by keeping features and labels independent
3. <u>WER</u>:  Word Error Rate is a metric used specifically for ASR models, which is used for evaluation

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


WER metric for analyzing ASR model - later

In [ ]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.6 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
# evaluate with the 'normalised' WER
do_normalize_eval = True

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]
        # filtering step to only evaluate the samples that correspond to non-zero references:
        pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
        label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Pretraining checkpoint, Configs, Training
1. WhisperForConditionalGeneration loads the checkpoints for Whisper-Medium
2. Kept a small batch size and gradient accumulation accordingly because of the 15GB limit on Colab
3. Using Seq2Seq trainer for training on Seq2SeqTrainingArguments
4. Then finally training the model on the hf dataset

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

In [ ]:

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=vectorized_datasets["train"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor
)

In [ ]:
model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

#References:
1. https://huggingface.co/docs/transformers/model_doc/whisper
2. https://openai.com/research/whisper
3. https://github.com/huggingface/transformers/blob/main/src/transformers/training_args_seq2seq.py
4. https://github.com/vasistalodagala/whisper-finetune/blob/master/train/fine-tune_on_hf_dataset.py

